In [ ]:
!nvidia-smi

Sun Dec 18 04:49:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 3.8 MB/s 
time: 380 µs (started: 2022-12-18 04:50:07 +00:00)


In [ ]:
!pip install nufhe[pycuda]
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 86 kB 2.3 MB/s 
     |████████████████████████████████| 191 kB 4.5 MB/s 
     |████████████████████████████████| 1.7 MB 11.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71 kB 8.4 MB/s 
     |████████████████████████████████| 78 kB 6.6 MB/s 
  Created wheel for pycuda: filename=pycuda-2022.2-cp38-cp38-linux_x86_64.whl size=646470 sha256=ea5d64a6e3c9b56e56228fd1cbbe2aa2734d8c82206dafa57dd7fcbf87d2d1da
  Stored in directory: /root/.cache/pip/wheels/ef/0e/11/809b20ee6df2e1c63ef52b3dcf2b7386b900d4f58548f0484e
  Created wheel for pytools: filename=pytools-2022.1.13-py2.py3-none-any.whl size=66024 sha256=05744cca97847d90a2606c090da0c2f77fbad2592539801279d761317a42c449
  Stored in directory: /root/.cache/pip/wheels/b5/c1/bb/26ba70f

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 30.9 s (started: 2022-12-18 04:52:14 +00:00)


In [ ]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime
def cba4(r,a,b,carry):
    temp= vm.gate_and(a, b) #ab
    temp1= vm.gate_and( b,carry ) #bc
    temp2= vm.gate_and(a, carry) # ac
    temp3= vm.gate_or(temp, temp1) #ab+ac
    r[1]=vm.gate_or(temp3,temp2) # ab+abc+ac
    r[0]=vm.gate_not(r[1]) #(ab+bc+ca)'
    return r
def cba1(r,a,b,carry):
    temp= vm.gate_and(b,carry) #temp=bc
    r[1]=vm.gate_or(temp, a)   #r1=bc+a
    r[0]=vm.gate_not(r[1])     #r0=(bc+a)'
    return r

def threetotwo(r,p0,p1,p2):
    r[0] = vm.gate_or(p0, p1)
    temp=vm.gate_and(p0, p1)
    r[1]=vm.gate_or(temp, p2)
    return r
def twotoone(r, p0,p1 ):
    r[0] = vm.gate_or(p0, p1)
    return r
def is_equal(a,b,nBits):
    #print("type of a",type(a))
    #print("type of b",type(b))
    t2= ctx.encrypt(secret_key, [False])
    #print("length of list",len(a))
    for i in range(0,nBits):
        t1=vm.gate_xor(a[i], b[i])
        t3=vm.gate_or(t2, t1)
        t2=vm.gate_copy(t3)
        equal=vm.gate_not(t3)

    return equal
def make_neg(n,nbits):
    list1=[int(i) for i in range(0,len(n)) ]
    listp=[]
    #one=vm.empty_ciphertext((1,))
    #zero=[vm.empty_ciphertext((1,))]
    #one= [ctx.encrypt(secret_key, [True])]
    #zero= [ctx.encrypt(secret_key, [False])]
    #t2= [ctx.encrypt(secret_key, [False])]
    #one=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    #zero=  [ctx.encrypt(secret_key, [False) for _ in range(nbits)]
    #print("type of one is",type(one))
    #print("typeof zero is",type(zero))
    #print("tupe of one is",one)
    #[vm.empty_ciphertext((1,))]
    '''for i in range(0,len(n)):
        #temp=vm.gate_copy(n[i])
        temp=vm.gate_mux(n[i],zero[0],one[0])
        n[i]=temp[:]'''


    #print(n)
    one= ctx.encrypt(secret_key, [True])
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    onep[0]=one
    testone= [[vm.empty_ciphertext((1,))] for i in range(nbits)]
    testone=onep[:]
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n before",twos_comp_val(int(pa,2),len(pa)))'''
    n=addNumbers(n,testone,nbits)
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n after",twos_comp_val(int(pa,2),len(pa)))'''
    return n


def nonRestoringDivision(m, q,nBits ):
    A=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    temp=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    mux1=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    mux2=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    neg2=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    q_c=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    mcopy=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    fullzero=[ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    one= ctx.encrypt(secret_key, [True])
    zero= ctx.encrypt(secret_key, [False])
    signbit= ctx.encrypt(secret_key, [False])
    #tw= ctx.encrypt(secret_key, [False])
    '''for i in range(len(q)):
        q_c=nufhe.LweSampleArray.copy(q[i])'''
    onep[0]=vm.gate_copy(one)
    c_zero= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    testone= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    ones= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    for i in range(nBits):
        c_zero[i]=vm.gate_copy(zero)
        #A[i]=vm.gate_copy(zero)
        testone[i]=vm.gate_copy(onep[i])
        q_c[i]=q[i]
        #A[i]=zero
    A=fullzero[:]
    mcopy=m[:]
    temp=c_zero[:]
    mux1=fullzero[:]
    equal_res= ctx.encrypt(secret_key, [False])
    for i in range(nBits):
      temp=vm.gate_copy(m[i])
      ones[i]=vm.gate_not(temp)
    neg2=make_neg(ones,nBits)
    temp2=neg2[:]
    temp2.reverse()
    result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("neg2 value",twos_comp_val(int(pa,2),len(pa)))
    count=len(m)
    print("********* loop started **************")

    while (True):
        print("===loop==",count)
        singbit=vm.gate_copy(A[len(A)-1])
        signplain=[]
        signplain.append([ctx.decrypt(secret_key,signbit)])
        print("signbit is",signplain)
        for i in range(len(A)-1,0,-1):
            A[i] =nufhe.LweSampleArray.copy(A[i-1])    #a=a>>1
        A[0]=vm.gate_copy(q_c[len(q_c)-1])             #a[0]=q[msb]
        for i in range(len(A)-1,0,-1):
            q_c[i] =vm.gate_copy(q_c[i-1])              #q=q>>1
        temp2=A[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("A after left shitf",twos_comp_val(int(pa,2),len(pa)))
        #aminusm=subtract(A,m,nBits)
        #aplusm=addNumbers(A,m,nBits)
        for i in range(0,nBits):
            mux1[i]=vm.gate_mux(signbit,mcopy[i],neg2[i])   #mux1= mcopy,neg2
        temp2=mcopy[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("m before add",twos_comp_val(int(pa,2),len(pa)))
        A=addNumbers(A,mux1,nBits)     #a=a+m or a-m
        temp2=A[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("A value loop",twos_comp_val(int(pa,2),len(pa)))
        #A=mux1[:]
        #equal_res=is_equal(A,c_zero,nBits)
        signbit=vm.gate_copy(A[len(A)-1])
        signplain1=[]
        signplain1.append([ctx.decrypt(secret_key,signbit)])
        print("@@@ signbit beforeupdate of q_C",signplain1)
        tw1=vm.gate_mux(signbit,zero,one)
        signplain2=[]
        signplain2.append([ctx.decrypt(secret_key,tw1)])
        print("@@@ tw1 beforeupdate of q_C",signplain2)
        q_c[0]=vm.gate_copy(tw1)   #Q[0]=select(A<zero,zero,one);
        #q_c.insert(0,tw1)
        #temp2 = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
        temp2=A[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("count is" ,count,"A inside loop",twos_comp_val(int(pa,2),len(pa)))
        temp2=q_c[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("count is" ,count,"q_c inside loop",twos_comp_val(int(pa,2),len(pa)))
        count -= 1
        if count==0:
            break
    #aminuszero=subtract(A,c_zero,nBits)
    signbit=vm.gate_copy(A[len(A)-1])
    aminuszero=A[:]
    aplusm =addNumbers(A,m,nBits)
    for i in range(0,nBits):
        A[i]=vm.gate_mux(signbit,aplusm[i],aminuszero[i])    #A=select(A<zero,A+M,A);
    return q_c
    #return A,q_c


def fixSizeBoolList(decimal,size):
    x = [int(x) for x in bin(decimal)[2:]]
    x = list(map(bool, x))
    x = [False]*(size - len(x)) + x
    pow2 = []
    for i in range(size):
      pow2.append([x[i]])
    pow2.reverse()
    return pow2
def boolListToInt(bitlists):
    out = 0
    for bit in bitlists:
        out = (out << 1) | bit
    return out

def addBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t5, t4)

    return r


def addNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
      if i>4:
        bitResult = addBits(bitResult, ctA[i], ctB[i], carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        # Copy(carry[0], bitResult[1])
        carry = nufhe.LweSampleArray.copy(bitResult[1])
      else:
        #bitResult = twotoone(bitResult, ctA[i], ctB[i])
        #bitResult = threetotwo(bitResult, ctA[i], ctB[i],carry)
        #bitResult =cba4(bitResult, ctA[i], ctB[i],carry)
        bitResult =cba1(bitResult, ctA[i], ctB[i],carry)
        #Copy(ctRes[i], bitResult[0]);

        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        carry = nufhe.LweSampleArray.copy(bitResult[1])

    return ctRes


def mulNumbers(ctA, ctB, secret_key, input_bits, output_bits):
    result = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]
    temp = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]
    # [[vm.empty_ciphertext((1,))] for _ in range(output_bits)]
    # andRes = [[vm.empty_ciphertext((1,))] for _ in range(input_bits)]

    for i in range(input_bits):
        andResLeft = [ctx.encrypt(secret_key, [False]) for _ in
                      range(output_bits)]
        #temp=mux(temp,ctA,ctB[i],size)
        for j in range(input_bits):
            andResLeft[j + i] = vm.gate_and(ctA[j], ctB[i])

            # andResLeft[j + i] = nufhe.LweSampleArray.copy(andRes[j])
        result = addNumbers(andResLeft, result, output_bits)
        #result_bits = [ctx.decrypt(secret_key, result[i]) for i in range(size * 2)]
        #print(" nuFHE multiplication intermdiate number is : ",i,boolListToInt(result_bits))

    return result[:size]
def Convert_list(string):
    list1=[]
    list1[:0]=string
    print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])

    #print(listb)
    return listb

def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}"
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s):
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')

    #print(listp)
    str1 = ""
    # traverse in the string
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s:
        str1 += ele
    # return string
    #print(str1)
    return str1
def twos_comp_val(val,bits):
    """compute the 2's complement of int value val"""
    #val=listToString(val)


    if (val & (1 << bits - 1)) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val
def compare_bit(  a, b,  lsb_carry,  tmp):
  result= ctx.encrypt(secret_key, [False])
  tmp=vm.gate_xnor(a, b)
  result=vm.gate_mux(tmp,lsb_carry, a)
  return result
def minimum(  a,  b,  nb_bits):
    tmps1= ctx.encrypt(secret_key, [False])
    tmps2= ctx.encrypt(secret_key, [True])
    #initialize the carry to 0
    #run the elementary comparator gate n times
    for i in range(nb_bits):
      tmps1= compare_bit(a[i],b[i],tmps1,tmps2)
    #tmps[0] is the result of the comparaison: 0 if a is larger, 1 if b is larger
    #select the max and copy it to the result
    return tmps1
def predict(ctA,secret_key, output_bits):
    zero = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    onen=  [ctx.encrypt(secret_key, [True]) for _ in range(output_bits)]
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    one= ctx.encrypt(secret_key, [True])
    temp= ctx.encrypt(secret_key, [True])
    temp=ctA[output_bits-1]
    comp_res= ctx.encrypt(secret_key, [True])
    onep[0]=one
    ctRes = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    # Copy(ctRes[i], bitResult[0]);
    #comp_res= minimum(ctA,zero,output_bits)
    #temp=
    #comp_res=
    for i in range(output_bits):
        ctRes[i] = vm.gate_mux(temp,onen[i],onep[i])
    # Copy(carry[0], bitResult[1])
    return ctRes
if __name__ == '__main__':
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    size=20
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    one= ctx.encrypt(secret_key, [True])
    zero=ctx.encrypt(secret_key, [False])
    onep[0]=one
    with open('/content/drive/MyDrive/train.txt') as f:
      lines = []
      for line in f:
        lines.append([int(v) for v in line.split()])

    bits = [[False] for i in range(size - 2)]
    zeros = [[False] for i in range(size)]

    test_size = int(input("Please enter a string:\n"))
    '''b_x0=[]
    b_x1=[]
    b_x2=[]
    b_x3=[]
    for i in range(test_size):
        temp=int(lines[i][0])
        #print(type(temp))
        b_x0.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][1])
        b_x1.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][2])
        b_x2.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][3])
        b_x3.append(fixSizeBoolList(temp,size))'''
    #print(b_x0,b_x1,b_x2,b_x3)
    #-3.5133618, -4.2616279 , 10.59763759,  6.17913037, -0.1758181
    #w_p=[-15,-18 ,-78 ,138 , 62] # scratch x/1+|x|
    w_p=[-1,-3,4,2,0] # sk learn x/1+|x|
    #w_p=[4,-8 ,22 ,9 , -1] sigmoid
    #w_p=[-3, -4 , 10,  6, -1] svm

    #print(type(x[0]))
    #print(x)
    #b_y = fixSizeBoolList(deci_y,size)
    #print(type(y[0]))
    #print(y)
    #x.reverse()
    #print(x)
    #y.reverse()
    #print(y)
    featuresize=5
    w=[]
    w_b=[]
    for i in range(featuresize-1):
        w.append([[vm.empty_ciphertext((1,))] for i in range(size)])
        w_b.append(twos_complement(w_p[i],size))
        for j in range(size):
            w[i][j] = ctx.encrypt(secret_key, w_b[i][j])
    bias=[[vm.empty_ciphertext((1,))] for i in range(size)]
    bias_b=twos_complement(w_p[4],size)
    eten=[[vm.empty_ciphertext((1,))] for i in range(size)]
    eten_b=twos_complement(10,size)
    for i in range(size):
        bias[i]=ctx.encrypt(secret_key, bias_b[i])
        eten[i]=ctx.encrypt(secret_key, eten_b[i])

    plain_predict=[]
    start_time = datetime.now()
    for i in range(test_size):
        print("i values is ",i)
        print(lines[i][0],lines[i][1],lines[i][2],lines[i][3])
        temp=int(lines[i][0])
        #print(type(temp))
        #x = twos_complement(deci_x,size)
        b_x0=twos_complement(temp,size)
        temp=int(lines[i][1])
        b_x1=twos_complement(temp,size)
        temp=int(lines[i][2])
        b_x2=twos_complement(temp,size)
        temp=int(lines[i][3])
        b_x3=twos_complement(temp,size)
        ciphertext1=[[vm.empty_ciphertext((1,))] for i in range(size)]

        ciphertext2=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext3=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext4=[[vm.empty_ciphertext((1,))] for i in range(size)]
        for j in range(size):
            ciphertext1[j] = ctx.encrypt(secret_key, b_x0[j])
            ciphertext2[j] = ctx.encrypt(secret_key, b_x1[j])
            ciphertext3[j] = ctx.encrypt(secret_key, b_x2[j])
            ciphertext4[j] = ctx.encrypt(secret_key, b_x3[j])
        #ciphertext1 = ctx.encrypt(secret_key, x)
        #ciphertext2 = ctx.encrypt(secret_key, y)
        start_time = time.time()
        #ciphertext1.reverse()
        #ciphertext2.reverse()
        #result = addNumbers(ciphertext1, ciphertext2, size)
        #partial_predict=[]

        #print(type(ciphertext1[0][0]))
        temp1 = [[vm.empty_ciphertext((1,))] for i in range(size)]
        temp2 = [[vm.empty_ciphertext((1,))] for i in range(size)]
        #wx=[[vm.empty_ciphertext((1,))] for i in range(size)]
        #predict[i]=[[vm.empty_ciphertext((1,))] for i in range(size*2)]
        #temp1=[]
        #temp2=[]
        presult_mul1 = mulNumbers(ciphertext1,w[0], secret_key, size, size * 2)
        #print("ciphertext one",type(ciphertext1))
        #print("ciphertext one inside",type(ciphertext1[0]))
        temp1=presult_mul1[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul1",twos_comp_val(int(pa,2),len(pa)))
        presult_mul2 = mulNumbers(ciphertext2, w[1], secret_key, size, size * 2)
        temp1=presult_mul2[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul2",twos_comp_val(int(pa,2),len(pa)))
        presult_mul3 = mulNumbers(ciphertext3, w[2], secret_key, size, size * 2)
        temp1=presult_mul3[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul3",twos_comp_val(int(pa,2),len(pa)))
        presult_mul4 = mulNumbers(ciphertext4, w[3], secret_key, size, size * 2)
        temp1=presult_mul4[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul4",twos_comp_val(int(pa,2),len(pa)))
        presult_add1 = addNumbers(presult_mul1, presult_mul2,  size)
        temp1=presult_add1[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("firstadd",pa)
        print("add1",twos_comp_val(int(pa,2),len(pa)))
        presult_add2 = addNumbers(presult_mul3, presult_mul4,  size)
        temp1=presult_add2[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        #print("secondadd",pa)
        print("add2",twos_comp_val(int(pa,2),len(pa)))
        wx= addNumbers(presult_add1, presult_add2,  size)
        temp1=wx[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("wx",twos_comp_val(int(pa,2),len(pa)))
        signbit=vm.gate_copy(wx[len(wx)-1])
        print("signbit is ",ctx.decrypt(secret_key, signbit))
        #print("wx type is",type(wx))
        #print("wx inside is",type(wx[0]))
        bits1 = [ctx.encrypt(secret_key,[False])] #[[vm.empty_ciphertext((1,))] for i in range(size)]
        #bits1=[[vm.empty_ciphertext((1,))] for i in range(1)]
        #print("bits1 datatype",type(bits1))
        wxcpy=[[vm.empty_ciphertext((1,))] for i in range(size)]
        wxcpy=wx[:]
        for i in range(size):
            bits3=vm.gate_copy(wx[i])
            wx[i]=vm.gate_not(bits3)
        wxtwos=make_neg(wx,size)
        x=[[vm.empty_ciphertext((1,))] for i in range(size)]

        for i in range(size):
            x[i]=vm.gate_mux(signbit,wxtwos[i],wxcpy[i])
        m=addNumbers(x,onep,size)
        temp1=m[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("m values is",twos_comp_val(int(pa,2),len(pa)))
        wxten=mulNumbers(wxcpy, eten, secret_key, size, size * 2)
        temp1=wxten[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("wxtenvalue",twos_comp_val(int(pa,2),len(pa)))
        sigmoid=nonRestoringDivision(m,wxten,size)
        temp1=sigmoid[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("divison result",twos_comp_val(int(pa,2),len(pa)))


        sigmoidneg = [[vm.empty_ciphertext((1,))] for i in range(size)]

        for i in range(size):
            bits1=vm.gate_copy(sigmoid[i])
            sigmoidneg[i]=vm.gate_not(bits1)
        sigmoidtwos=addNumbers(sigmoidneg,onep,size)
        temp1=sigmoidtwos[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("sigmoid2s",twos_comp_val(int(pa,2),len(pa)))
        fsigmoid = [[vm.empty_ciphertext((1,))] for i in range(size)]
        for i in range(size):
            fsigmoid[i]=vm.gate_mux(signbit,sigmoidtwos[i],sigmoid[i])

        temp1=fsigmoid[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        #print("fsigmoid",pa)
        print("fsigmoid",twos_comp_val(int(pa,2),len(pa)))
        #print("fsigmoid",len(fsigmoid))
        '''temp1=presult_add3[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("thirdadd",pa)
        print("add3",twos_comp_val(int(pa,2),len(pa)))'''
        #partial_predict = addNumbers(presult_add3, bias,  size)
        #partial_predict.revesrse()
        '''result_bits = [ctx.decrypt(secret_key, partial_predict[i]) for i in range(size)]
        result_bits.reverse()
        pa=listToString(result_bits)
        print("partial predict",twos_comp_val(int(pa,2),len(pa)))'''

        predict1=predict(fsigmoid,secret_key,size)
        predict1.reverse()
        #print(predict1)
        #print(type(predict1))
        #print(type(predict1[0]))
        #result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size*2)]
        result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size)]
        #plain_predict.append(boolListToInt(result_bits))
        pa=listToString(result_bits)
        plain_predict.append(twos_comp_val(int(pa,2),len(pa)))
        #print(" nuFHE multiplication number is : ", plain_predict[i])
        print("i value is",i, "endtime",datetime.now(),"start time",start_time)
    #print("start time is",start_time)
    #print("end time",time.time())
    print("endtime",datetime.now(),"start time",start_time)
    #result.reverse()
    for j in range(test_size):
        print(" nuFHE multiplication number is actual",lines[j][4],"preicted", plain_predict[j])

RuntimeError: ignored

time: 69.7 ms (started: 2022-12-16 23:11:10 +00:00)


In [ ]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime


def threetotwo(r,p0,p1,p2):
  r[0] = vm.gate_or(p0, p1)
  temp=vm.gate_and(p0, p1)
  r[1]=vm.gate_or(temp, p2)
  return r
def twotoone(r, p0,p1 ):
  r[0] = vm.gate_or(p0, p1)
  return r
def is_equal(a,b,nBits):
    #print("type of a",type(a))
    #print("type of b",type(b))
    t2= ctx.encrypt(secret_key, [False])
    #print("length of list",len(a))
    for i in range(0,nBits):
        t1=vm.gate_xor(a[i], b[i])
        t3=vm.gate_or(t2, t1)
        t2=vm.gate_copy(t3)
        equal=vm.gate_not(t3)

    return equal
def make_neg(n,nbits):
    list1=[int(i) for i in range(0,len(n)) ]
    listp=[]
    one= ctx.encrypt(secret_key, [True])
    zero= ctx.encrypt(secret_key, [False])
    t2= ctx.encrypt(secret_key, [False])
    #print(n)
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    onep[0]=one
    testone= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    testone=onep[:]
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n before",twos_comp_val(int(pa,2),len(pa)))'''
    n=addNumbers(n,testone,nBits)
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n after",twos_comp_val(int(pa,2),len(pa)))'''
    return n


def nonRestoringDivision(m, q,nBits ):
    A=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    temp=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    mux1=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    mux2=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    neg2=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    q_c=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    mcopy=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    fullzero=[ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    #one=[vm.empty_ciphertext((1,))]
    #zero=[vm.empty_ciphertext((1,))]
    one= ctx.encrypt(secret_key, [True])
    zero= ctx.encrypt(secret_key, [False])
    signbit= ctx.encrypt(secret_key, [False])
    #tw= ctx.encrypt(secret_key, [False])
    '''for i in range(len(q)):
        q_c=nufhe.LweSampleArray.copy(q[i])'''
    onep[0]=vm.gate_copy(one)
    c_zero= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    testone= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    for i in range(nBits):
        c_zero[i]=vm.gate_copy(zero)
        #A[i]=vm.gate_copy(zero)
        testone[i]=vm.gate_copy(onep[i])
        q_c[i]=q[i]
        #A[i]=zero
    A=fullzero[:]
    mcopy=m[:]
    temp=c_zero[:]
    mux1=fullzero[:]
    equal_res= ctx.encrypt(secret_key, [False])
    neg2=make_neg(m,nBits)
    temp2=neg2[:]
    temp2.reverse()
    result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("neg2 value",twos_comp_val(int(pa,2),len(pa)))
    count=16
    print("********* loop started **************")
    while (count):
        print("===loop==",count)
        singbit=vm.gate_copy(A[len(A)-1])
        signplain=[]
        signplain.append([ctx.decrypt(secret_key,signbit)])
        print("signbit is",signplain)
        for i in range(len(A)-1,0,-1):
            A[i] =nufhe.LweSampleArray.copy(A[i-1])
        A[0]=vm.gate_copy(q_c[len(q_c)-1])
        for i in range(len(A)-1,0,-1):
            q_c[i] =vm.gate_copy(q_c[i-1])
        temp2=A[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("A after left shitf",twos_comp_val(int(pa,2),len(pa)))
        #aminusm=subtract(A,m,nBits)
        #aplusm=addNumbers(A,m,nBits)
        for i in range(0,nBits):
            mux1[i]=vm.gate_mux(signbit,mcopy[i],neg2[i])
        temp2=mcopy[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("m before add",twos_comp_val(int(pa,2),len(pa)))
        A=addNumbers(A,mux1,nBits)
        temp2=A[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("A value loop",twos_comp_val(int(pa,2),len(pa)))
        #A=mux1[:]
        #equal_res=is_equal(A,c_zero,nBits)
        signbit=vm.gate_copy(A[len(A)-1])
        signplain1=[]
        signplain1.append([ctx.decrypt(secret_key,signbit)])
        #print("@@@ signbit beforeupdate of q_C",signplain1)
        tw1=vm.gate_mux(signbit,zero,one)
        signplain2=[]
        signplain2.append([ctx.decrypt(secret_key,tw1)])
        #print("@@@ tw1 beforeupdate of q_C",signplain2)
        q_c[0]=vm.gate_copy(tw1)
        #q_c.insert(0,tw1)
        #temp2 = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
        temp2=A[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("count is" ,count,"A inside loop",twos_comp_val(int(pa,2),len(pa)))
        temp2=q_c[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("count is" ,count,"q_c inside loop",twos_comp_val(int(pa,2),len(pa)))
        count -= 1
    #aminuszero=subtract(A,c_zero,nBits)
    signbit=vm.gate_copy(A[len(A)-1])
    aminuszero=A[:]
    aplusm =addNumbers(A,c1,nBits)
    for i in range(0,nBits):
        A[i]=vm.gate_mux(signbit,aplusm[i],aminuszero[i])
    return q_c
    #return A,q_c


def fixSizeBoolList(decimal,size):
    x = [int(x) for x in bin(decimal)[2:]]
    x = list(map(bool, x))
    x = [False]*(size - len(x)) + x
    pow2 = []
    for i in range(size):
      pow2.append([x[i]])
    pow2.reverse()
    return pow2
def boolListToInt(bitlists):
    out = 0
    for bit in bitlists:
        out = (out << 1) | bit
    return out

def addBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t5, t4)

    return r


def addNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
      if i>4:
        bitResult = addBits(bitResult, ctA[i], ctB[i], carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        # Copy(carry[0], bitResult[1])
        carry = nufhe.LweSampleArray.copy(bitResult[1])
    else:
        bitResult = twotoone(bitResult, ctA[i], ctB[i])
        #bitResult = threetotwo(bitResult, ctA[i], ctB[i],carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        #carry = nufhe.LweSampleArray.copy(bitResult[1])


    return ctRes


def mulNumbers(ctA, ctB, secret_key, input_bits, output_bits):
    result = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]
    temp = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]
    # [[vm.empty_ciphertext((1,))] for _ in range(output_bits)]
    # andRes = [[vm.empty_ciphertext((1,))] for _ in range(input_bits)]

    for i in range(input_bits):
        andResLeft = [ctx.encrypt(secret_key, [False]) for _ in
                      range(output_bits)]
        #temp=mux(temp,ctA,ctB[i],size)
        for j in range(input_bits):
            andResLeft[j + i] = vm.gate_and(ctA[j], ctB[i])

            # andResLeft[j + i] = nufhe.LweSampleArray.copy(andRes[j])
        result = addNumbers(andResLeft, result, output_bits)
        #result_bits = [ctx.decrypt(secret_key, result[i]) for i in range(size * 2)]
        #print(" nuFHE multiplication intermdiate number is : ",i,boolListToInt(result_bits))

    return result[:size]
def Convert_list(string):
    list1=[]
    list1[:0]=string
    print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])

    #print(listb)
    return listb

def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}"
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s):
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')

    #print(listp)
    str1 = ""
    # traverse in the string
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s:
        str1 += ele
    # return string
    #print(str1)
    return str1
def twos_comp_val(val,bits):
    """compute the 2's complement of int value val"""
    #val=listToString(val)


    if (val & (1 << bits - 1)) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val
def compare_bit(  a, b,  lsb_carry,  tmp):
  result= ctx.encrypt(secret_key, [False])
  tmp=vm.gate_xnor(a, b)
  result=vm.gate_mux(tmp,lsb_carry, a)
  return result
def minimum(  a,  b,  nb_bits):
    tmps1= ctx.encrypt(secret_key, [False])
    tmps2= ctx.encrypt(secret_key, [True])
    #initialize the carry to 0
    #run the elementary comparator gate n times
    for i in range(nb_bits):
      tmps1= compare_bit(a[i],b[i],tmps1,tmps2)
    #tmps[0] is the result of the comparaison: 0 if a is larger, 1 if b is larger
    #select the max and copy it to the result
    return tmps1
def predict(ctA,secret_key, output_bits):
    zero = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    onen=  [ctx.encrypt(secret_key, [True]) for _ in range(output_bits)]
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    one= ctx.encrypt(secret_key, [True])
    temp= ctx.encrypt(secret_key, [True])
    temp=ctA[output_bits-1]
    comp_res= ctx.encrypt(secret_key, [True])
    onep[0]=one
    ctRes = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    # Copy(ctRes[i], bitResult[0]);
    #comp_res= minimum(ctA,zero,output_bits)
    #temp=
    #comp_res=
    for i in range(output_bits):
      ctRes[i] = vm.gate_mux(temp,onen[i],onep[i])
    # Copy(carry[0], bitResult[1])
    return ctRes
if __name__ == '__main__':
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    size=16
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    one= ctx.encrypt(secret_key, [True])
    onep[0]=one
    with open('/content/drive/MyDrive/train.txt') as f:
      lines = []
      for line in f:
        lines.append([int(v) for v in line.split()])

    bits = [[False] for i in range(size - 2)]
    zeros = [[False] for i in range(size)]

    test_size = int(input("Please enter a string:\n"))
    '''b_x0=[]
    b_x1=[]
    b_x2=[]
    b_x3=[]
    for i in range(test_size):
        temp=int(lines[i][0])
        #print(type(temp))
        b_x0.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][1])
        b_x1.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][2])
        b_x2.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][3])
        b_x3.append(fixSizeBoolList(temp,size))'''
    #print(b_x0,b_x1,b_x2,b_x3)
    #-3.5133618, -4.2616279 , 10.59763759,  6.17913037, -0.1758181
    w_p=[4,-8 ,22 ,9 , -1]
    #w_p=[-3, -4 , 10,  6, -1]

    #print(type(x[0]))
    #print(x)
    #b_y = fixSizeBoolList(deci_y,size)
    #print(type(y[0]))
    #print(y)
    #x.reverse()
    #print(x)
    #y.reverse()
    #print(y)
    featuresize=5
    w=[]
    w_b=[]
    for i in range(featuresize-1):
        w.append([[vm.empty_ciphertext((1,))] for i in range(size)])
        w_b.append(twos_complement(w_p[i],size))
        for j in range(size):
            w[i][j] = ctx.encrypt(secret_key, w_b[i][j])
    bias=[[vm.empty_ciphertext((1,))] for i in range(size)]
    bias_b=twos_complement(w_p[4],size)
    eten=[[vm.empty_ciphertext((1,))] for i in range(size)]
    eten_b=twos_complement(10,size)
    for i in range(size):
      bias[i]=ctx.encrypt(secret_key, bias_b[i])
      eten[i]=ctx.encrypt(secret_key, eten_b[i])

    plain_predict=[]
    start_time = datetime.now()
    for i in range(test_size):
        print(lines[i][0],lines[i][1],lines[i][2],lines[i][3])
        temp=int(lines[i][0])
        #print(type(temp))
        #x = twos_complement(deci_x,size)
        b_x0=twos_complement(temp,size)
        temp=int(lines[i][1])
        b_x1=twos_complement(temp,size)
        temp=int(lines[i][2])
        b_x2=twos_complement(temp,size)
        temp=int(lines[i][3])
        b_x3=twos_complement(temp,size)
        ciphertext1=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext2=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext3=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext4=[[vm.empty_ciphertext((1,))] for i in range(size)]
        for j in range(size):
            ciphertext1[j] = ctx.encrypt(secret_key, b_x0[j])
            ciphertext2[j] = ctx.encrypt(secret_key, b_x1[j])
            ciphertext3[j] = ctx.encrypt(secret_key, b_x2[j])
            ciphertext4[j] = ctx.encrypt(secret_key, b_x3[j])
        #ciphertext1 = ctx.encrypt(secret_key, x)
        #ciphertext2 = ctx.encrypt(secret_key, y)
        start_time = time.time()
        #ciphertext1.reverse()
        #ciphertext2.reverse()
        #result = addNumbers(ciphertext1, ciphertext2, size)
        #partial_predict=[]

        #print(type(ciphertext1[0][0]))
        temp1 = [[vm.empty_ciphertext((1,))] for i in range(size)]
        temp2 = [[vm.empty_ciphertext((1,))] for i in range(size)]
        #predict[i]=[[vm.empty_ciphertext((1,))] for i in range(size*2)]
        #temp1=[]
        #temp2=[]
        presult_mul1 = mulNumbers(ciphertext1,w[0], secret_key, size, size * 2)
        '''temp1=presult_mul1[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul1",twos_comp_val(int(pa,2),len(pa)))'''
        presult_mul2 = mulNumbers(ciphertext2, w[1], secret_key, size, size * 2)
        '''temp1=presult_mul2[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul2",twos_comp_val(int(pa,2),len(pa)))'''
        presult_mul3 = mulNumbers(ciphertext3, w[2], secret_key, size, size * 2)
        '''temp1=presult_mul3[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul3",twos_comp_val(int(pa,2),len(pa)))'''
        presult_mul4 = mulNumbers(ciphertext4, w[3], secret_key, size, size * 2)
        '''temp1=presult_mul4[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul4",twos_comp_val(int(pa,2),len(pa)))'''
        presult_add1 = addNumbers(presult_mul1, presult_mul2,  size)
        '''temp1=presult_add1[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("firstadd",pa)
        print("add1",twos_comp_val(int(pa,2),len(pa)))'''
        presult_add2 = addNumbers(presult_mul3, presult_mul4,  size)
        '''temp1=presult_add2[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("secondadd",pa)
        print("add2",twos_comp_val(int(pa,2),len(pa)))'''

        presult_add2= addNumbers(presult_add1, presult_add2,  size)
        wx=presult_add2[:]
        signbit=vm.gate_copy(wx[len(wx)-1])
        for i in range(size):
            #temp=wx[i]
            wx[i]=vm.gate_not(wx[i])
            #wx[i]=temp
        wxtwos=make_neg(wx,size)

        for i in range(len(wx)):
          x[i]=vm.vm.gate_mux(signbit,wxtwos[i],wx[i])
        m=addNumbers(x,onep,size)
        wxten=mulNumbers(x, eten, secret_key, size, size * 2)

        sigmoid=nonRestoringDivision(m,wxten,size)


        '''temp1=presult_add3[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("thirdadd",pa)
        print("add3",twos_comp_val(int(pa,2),len(pa)))'''
        #partial_predict = addNumbers(presult_add3, bias,  size)
        #partial_predict.revesrse()
        '''result_bits = [ctx.decrypt(secret_key, partial_predict[i]) for i in range(size)]
        result_bits.reverse()
        pa=listToString(result_bits)
        print("partial predict",twos_comp_val(int(pa,2),len(pa)))'''
        predict1=predict(sigmoid,secret_key,size)
        predict1.reverse()
        #print(predict1)
        #print(type(predict1))
        #print(type(predict1[0]))
        #result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size*2)]
        result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size)]
        #plain_predict.append(boolListToInt(result_bits))
        pa=listToString(result_bits)
        plain_predict.append(twos_comp_val(int(pa,2),len(pa)))
        print(" nuFHE multiplication number is : ", plain_predict[i])
    #print("start time is",start_time)
    #print("end time",time.time())
    print("endtime",datetime.now(),"start time",start_time)
    #result.reverse()
    for j in range(test_size):
        print(" nuFHE multiplication number is actual",lines[j][4],"preicted", plain_predict[j])


Please enter a string:
2
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0']
51 25 30 11
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '1']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1']


AttributeError: ignored

time: 5min 27s (started: 2021-10-19 16:57:51 +00:00)
